In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
from datetime import date

import re # for regular expressions
import string
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
import advertools as adv

In [3]:
# read tweets from csv file
tweets = pd.read_csv('./data/0401_UkraineCombinedTweetsDeduped.csv')
tweets = tweets[tweets['language']=="en"]
tweets = tweets[["tweetcreatedts", "text"]]
tweets.head()

C:\Users\51588\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tweetcreatedts,text
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...


In [4]:
# for test propose, reduce records
tweets_short = tweets[0:1000]

In [7]:
from nrclex import NRCLex
from collections import Counter
from tqdm import tqdm
from Lexicon import lexicon
tqdm.pandas()
class EmotionDetector:
    # copyed from nrclex.py, remove positive and negative emotions

    wordnet_lemmatizer = WordNetLemmatizer()
    tk = TweetTokenizer()
    stop_words = set(stopwords.words('english'))
    # initialize lexicon
    def __init__(self, emoji_factor = 1):
        # load lexico from https://github.com/abushoeb/EmoTag for emoji emotion detection
        #ncr = pd.read_csv('resource_folder/lexicon.csv', sep =';')
        emoji_lexicon = pd.read_csv('resource_folder/EmoTag1200-scores.csv', sep =',')
        self.emoji_dict = emoji_lexicon.set_index('emoji').T.to_dict()
        self.emoji_factor = emoji_factor

    def process(self, dataFrame, columnName='text'):
        print("begin extract emoji ")
        emoji_summary = adv.extract_emoji(dataFrame[columnName])
        dataFrame['emojis'] = emoji_summary['emoji']
        #dataFrame['tokens'] = dataFrame[columnName].apply(EmotionDetector.pre_process)
        dataFrame['tokens'] = dataFrame[columnName].progress_apply(EmotionDetector.pre_process)
        res = dataFrame.progress_apply(self.emotion_analysis,axis=1)
        result = pd.concat([dataFrame, res], axis=1)
        return result

    def emotion_analysis(self, row):
        #print("1111")
        #print(row)
        words = row["tokens"]
        emojis = row['emojis']
        # refer NRCLex
        affect_list = []
        affect_dict = dict()
        affect_frequencies = Counter()
        lexicon_keys = lexicon.keys()
        for word in words:
            if word in lexicon_keys:
                affect_list.extend(lexicon[word])
                affect_dict.update({word: lexicon[word]})
        for word in affect_list:
            affect_frequencies[word] += 1



        affect_percent = {'fear': 0.0, 'anger': 0.0, 'anticipation': 0.0, 'trust': 0.0, 'surprise': 0.0,
                           'sadness': 0.0, 'disgust': 0.0, 'joy': 0.0}

        emoji_lexicon_keys = self.emoji_dict.keys();
        for emoji in emojis:
            #print(emoji)
            if emoji in emoji_lexicon_keys:
                emoji_affect_dic = self.emoji_dict[emoji]
                for key in affect_percent.keys():
                    if emoji_affect_dic[key] > 0:
                        affect_frequencies[key] += emoji_affect_dic[key] * self.emoji_factor
                        affect_list.append(key)
                        #affect_dict.update({emoji: {affect:emoji_affect_dic[affect] for affect in emoji_affect_dic if emoji_affect_dic[affect] >0}})
        sum_values = sum(affect_frequencies.values())
        for key in affect_frequencies.keys():
            #print(affect_frequencies)
            affect_percent.update({key: float(affect_frequencies[key]) / float(sum_values)})
        emo_dict = affect_frequencies
        top_emotions = []
        if emo_dict.__len__()>0:
            max_value = max(emo_dict.values())
            for key in emo_dict.keys():
                if emo_dict[key] == max_value:
                    top_emotions.append((key, max_value))

        return pd.Series([affect_list,affect_dict,dict(affect_frequencies),affect_percent,top_emotions],
                         index=['affect_list', 'affect_dict', 'affect_frequencies', 'affect_percent','top_emotions'])

    @staticmethod
    def pre_process(text):
        #print(text)
        # clean text
        text = EmotionDetector.cleanText(str(text))
        #print(text)
        #emoji_summary = adv.extract_emoji([text])
        #print(emoji_summary['emoji'])
        #tokenization = nltk.word_tokenize(text)
        tokenization = EmotionDetector.tk.tokenize(text)

        filtered_words = [word for word in tokenization if word not in EmotionDetector.stop_words]
        map_object = map(EmotionDetector.wordnet_lemmatizer.lemmatize, filtered_words)
        lemma_words = list(map_object)
        #print(lemma_words)
        return lemma_words


    @staticmethod
    def cleanText(text):
        text=text.lower()
        text = re.sub(r'@\w+','', text) # remove @member
        text = re.sub(r'#','', text) # remove # symbol
        text = re.sub(r'https?:\/\/\S+','', text) # remove the hyper link
        return text

In [8]:
# test
ed = EmotionDetector()
res = ed.process(tweets_short)
res.head(100)

begin extract emoji 


<ipython-input-7-8c4d4ad2d5c8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['emojis'] = emoji_summary['emoji']
100%|██████████| 1000/1000 [00:00<00:00, 1698.12it/s]
<ipython-input-7-8c4d4ad2d5c8>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['tokens'] = dataFrame[columnName].progress_apply(EmotionDetector.pre_process)
100%|██████████| 1000/1000 [00:00<00:00, 1514.81it/s]


,tweetcreatedts,text,emojis,tokens,affect_list,affect_dict,affect_frequencies,affect_percent,top_emotions
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, 🇺🇦, 🧵]","[⚡, ukrainian, air, force, would, like, addres...","[anger, fear, trust, fear, anger, anticipation...","{'force': ['anger', 'fear'], 'ally': ['trust']}","{'anger': 1.28, 'fear': 1.25, 'trust': 1.14000...","{'fear': 0.2790178571428571, 'anger': 0.285714...","[(anger, 1.28)]"
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,[],"[chernihiv, oblast, ., ukrainian, welcome, lib...",[],{},{},"{'fear': 0.0, 'anger': 0.0, 'anticipation': 0....",[]
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,"[🇺🇸, 🇹🇼, 🇷🇺, 🇺🇸, 🇨🇳, 👇]","[america, 🇺, 🇸, preparing, something, worse, r...","[fear, sadness, fear, trust, fear, anger, anti...","{'worse': ['fear', 'sadness'], 'war': ['fear']...","{'fear': 2.06, 'sadness': 1, 'trust': 1.11, 'a...","{'fear': 0.4660633484162896, 'anger': 0.024886...","[(fear, 2.06)]"
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,[],"[:, anonymous, hacked, &, released, 62,000, em...",[],{},{},"{'fear': 0.0, 'anger': 0.0, 'anticipation': 0....",[]
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],"[*, *, *, public, mint, live, *, *, *, win, $,...","[anticipation, anticipation, anticipation, joy...","{'public': ['anticipation'], 'good': ['anticip...","{'anticipation': 4, 'joy': 2, 'surprise': 2, '...","{'fear': 0.0, 'anger': 0.0, 'anticipation': 0....","[(anticipation, 4)]"
...,...,...,...,...,...,...,...,...,...
123,2022-04-01 00:00:30.000000,💙💛 Cover of the April issue of Polish Vogue \...,"[💙, 💛]","[💙, 💛, cover, april, issue, polish, vogue, cre...","[trust, anticipation, trust, surprise, sadness...",{'cover': ['trust']},"{'trust': 1.6400000000000001, 'anticipation': ...","{'fear': 0.0, 'anger': 0.00804289544235925, 'a...","[(trust, 1.6400000000000001)]"
124,2022-04-01 00:00:30.000000,#Ukrainian forces successfully conducted local...,[],"[ukrainian, force, successfully, conducted, lo...","[anger, fear, anticipation]","{'force': ['anger', 'fear'], 'coming': ['antic...","{'anger': 1, 'fear': 1, 'anticipation': 1}","{'fear': 0.3333333333333333, 'anger': 0.333333...","[(anger, 1), (fear, 1), (anticipation, 1)]"
125,2022-04-01 00:00:31.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, 🇺🇦, 🧵]","[⚡, ukrainian, air, force, would, like, addres...","[anger, fear, trust, fear, anger, anticipation...","{'force': ['anger', 'fear'], 'ally': ['trust']}","{'anger': 1.28, 'fear': 1.25, 'trust': 1.14000...","{'fear': 0.2790178571428571, 'anger': 0.285714...","[(anger, 1.28)]"
127,2022-04-01 00:00:31.000000,❌Myth: The US and @NATO are providing #Ukraine...,"[❌, ✅]","[❌, myth, :, u, providing, ukraine, needed, we...","[anticipation, joy, trust, trust, anger, fear,...","{'providing': ['anticipation', 'joy', 'trust']...","{'anticipation': 2.45, 'joy': 1.19, 'trust': 4...","{'fear': 0.16363636363636364, 'anger': 0.18181...","[(trust, 4.39)]"


In [134]:
res.to_csv("res.csv")

,tweetcreatedts,text,tokens,emoji
0,2022-04-01 00:00:00.000000,⚡The Ukrainian Air Force would like to address...,"[⚡, ukrainian, air, force, would, like, addres...","[⚡, 🇺🇦, 🧵]"
1,2022-04-01 00:00:00.000000,Chernihiv oblast. Ukrainians welcome their lib...,"[chernihiv, oblast, ., ukrainian, welcome, lib...",[]
2,2022-04-01 00:00:00.000000,America 🇺🇸 is preparing for something worse th...,"[america, 🇺, 🇸, preparing, something, worse, r...","[🇺🇸, 🇹🇼, 🇷🇺, 🇺🇸, 🇨🇳, 👇]"
3,2022-04-01 00:00:00.000000,JUST IN: #Anonymous has hacked &amp; released ...,"[:, anonymous, hacked, &, released, 62,000, em...",[]
4,2022-04-01 00:00:00.000000,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,"[*, *, *, public, mint, live, *, *, *, win, $,...",[]
5,2022-04-01 00:00:00.000000,The Amazing story of Former Sussex County Dela...,"[amazing, story, former, sussex, county, delaw...",[🇺🇸]
6,2022-04-01 00:00:00.000000,&amp;quot;How we were waiting for you!&amp;quo...,"[&, quot, ;, waiting, !, &, quot, ;:, chernihi...",[]
7,2022-04-01 00:00:00.000000,India's purchase of discounted #Russian crude ...,"[india's, purchase, discounted, russian, crude...",[]
8,2022-04-01 00:00:00.000000,The most basic tenet of what stability exists ...,"[basic, tenet, stability, exists, world, borde...",[]
9,2022-04-01 00:00:00.000000,"""The image that Russia acquired over the past ...","["", image, russia, acquired, past, two, decade...",[]
